In [2]:
import tensorflow as tf
try:
    tf.get_logger().setLevel('INFO')
except Exception as exc:
    print(exc)
import warnings
warnings.simplefilter("ignore")

import numpy as np
import torch
import torch.optim as optim
from torchvision import datasets
from ray.tune.examples.mnist_pytorch import train, test, ConvNet, get_data_loaders

import ray
from ray import tune
from ray.tune import track
from ray.tune.suggest import HyperOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.style as style
style.use("ggplot")

datasets.MNIST("~/data", train=True, download=True)

Dataset MNIST
    Number of datapoints: 60000
    Root location: /Users/rliaw/data
    Split: Train

In [2]:
def train_mnist(config):
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model = ConvNet().to(device)
    train_loader, test_loader = get_data_loaders()

    optimizer = optim.SGD(
        model.parameters(), lr=config["lr"], momentum=config["momentum"])

    for i in range(100):
        train(model, optimizer, train_loader, device)  # Train for 1 epoch
        acc = test(model, test_loader, device)  # Obtain validation accuracy.
        tune.track.log(mean_accuracy=acc)
        if i % 5 == 0:
            torch.save(model, "./model.pth") # This saves the model to the trial directory

In [3]:
train_mnist(config={"lr": 0.1, "momentum": 0.01})

## Demonstrate a grid search

In [ ]:
tune.run(
    train_mnist,
    name="grid",
    config={"lr": tune.loguniform(0.0001, 0.1), "momentum": tune.uniform(0.1, 0.9)},
    num_samples=20,
    verbose=1
)

## Demonstrate using HyperOpt

In [ ]:
space = {
    "lr": hp.loguniform("lr", -10, -1),  # base e
    "momentum": hp.uniform("momentum", 0.1, 0.9),
}


In [ ]:
tune.run(
    train_mnist,
    name="search_algorithm",
    num_samples=20,
    search_alg=HyperOptSearch(space=space, max_concurrent=1, metric="mean_accuracy", mode="max"),
    verbose=1
)

In [4]:
tune.run(
    train_mnist,
    name="asha",
    config={"lr": tune.loguniform(0.0001, 0.1), "momentum": tune.uniform(0.1, 0.9)},
    scheduler=AsyncHyperBandScheduler(metric="mean_accuracy", mode="max"),
    verbose=1
)

SyntaxError: invalid syntax (<ipython-input-4-38045f6d660f>, line 6)

%load_ext tensorboard
%tensorboard --logdir ~/ray_results/train_mnist